In [6]:
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
sys.path.insert(0, '..')
from src.embedding import Embedding
from src.forward_kinematics import ForwardKinematic
from src.dynamical_system import DynamicalSystem

In [11]:
fk = ForwardKinematic(urdf_file='../description/2_joints_arm.urdf', gmm_configuration_file="../config/gmm_unit.json")

joint_limits = [{'lower': -np.pi/2, 'upper': np.pi/2}, {'lower': -np.pi/2, 'upper': np.pi/2}]
e = Embedding(dimension=2, x=x.repeat(1, 0), fk=fk, limits=joint_limits)

ds = DynamicalSystem(stiffness=K, dissipation=D, attractor=config_attractor, embedding=e, dt=0.01)

FileNotFoundError: [Errno 2] No such file or directory: 'franka_description/meshes/refined/visual_link0.stl'